# Исследование надежности заемщиков


Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Обзор данных

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
try:
    data = pd.read_csv('/datasets/data.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')

In [ ]:
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB




---


**В данных есть некоторые проблемы, а именно пропуски, неправильные форматы данных. Необходимо будет устранить эти проблемы, поработав с данными**

---



## Предобработка данных

### Удаление пропусков

Подсчитаем пропуски

In [ ]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64



---


В одной из колонок имеются отсутствующие значения. Мы обработаем пропущенные значения в этой колонке на следующем этапе. Вторая колонка с пропущенными значениями называется `total_income` и содержит информацию о доходах. Тип занятости сильно влияет на общую сумму дохода, поэтому мы заполним пропущенные значения в этой колонке медианным значением для каждого типа занятости из колонки `income_type`. Например, для человека с типом занятости "сотрудник" пропущенное значение в колонке `total_income` будет заполнено медианным доходом среди всех записей с таким же типом занятости.

---



In [ ]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### Обработка аномальных значений

**В данных могут встречаться артефакты (аномалии) — значения, которые не отражают действительность и появились по какой-то ошибке. таким артефактом будет отрицательное количество дней трудового стажа в столбце `days_employed`.**

In [ ]:
data['days_employed'] = data['days_employed'].abs()

**Для каждого типа занятости выведите медианное значение трудового стажа `days_employed` в днях.**

In [ ]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, поэтому оставьте их как есть. Тем более этот столбец не понадобится вам для исследования.

**Посмотрим на уникальные значения столбца `children`.**

In [ ]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

**В столбце `children` есть два аномальных значения. Удалиv строки, в которых встречаются такие аномальные значения из датафрейма `data`.**

In [ ]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

**Проверяем**

In [ ]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5])

**Заполним пропуски в столбце days_employed медианными значениями по каждого типа занятости `income_type`.**

In [ ]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

**Проверим**

In [ ]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Изменение типов данных

**Заменим вещественный тип данных в столбце `total_income` на целочисленный с помощью метода `astype()`.**

In [ ]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

**Обработайте неявные дубликаты в столбце `education`. В этом столбце есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв.**

In [ ]:
data['education'] = data['education'].str.lower()

**Посчитаем строки-дубликаты**

In [ ]:
data.duplicated().sum()

71

In [ ]:
data = data.drop_duplicates()

### Категоризация данных

**Задание 16. На основании диапазонов, указанных ниже, создадим в датафрейме `data` столбец `total_income_category` с категориями:**

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.


In [ ]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [ ]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

**Выведим на экран перечень уникальных целей взятия кредита из столбца `purpose`.**

In [ ]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

**Создадим функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:**

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.


In [ ]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [ ]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

### Шаг 3. Исследуйте данные и ответьте на вопросы

#### 3.1 Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [ ]:
# Изучаем данные
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,B,операции с недвижимостью
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,B,операции с недвижимостью
7,0,152.779569,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,C,получение образования
8,2,6929.865299,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,C,проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,C,операции с недвижимостью


In [ ]:
#построим сводную таблицу зависимости возврата кредита и количество детей
children_and_kredit = data.pivot_table(columns = 'children', values = 'debt')
children_and_kredit

children,0,1,2,3,4,5
debt,0.075438,0.092346,0.094542,0.081818,0.097561,0.0


In [ ]:
one_child = data.loc[data['children'] == 1]
print('Количество семей с одним ребенком', len(one_child))

In [ ]:
two_children = data.loc[data['children'] == 2]
print('Количество семей с двумя детьми ', len(two_children))

Количество семей с двумя детьми  2052


In [ ]:
three_children = data.loc[data['children'] == 3]
print('Количество семей с тремя детьми', len(three_children))

Количество семей с тремя детьми 330


In [ ]:
four_children = data.loc[data['children'] == 4]
print('Количество семей с четырмя детьми', len(four_children))

Количество семей с четырмя детьми 41


In [ ]:
five_children = data.loc[data['children'] == 5]
print('Количество семей с пятью детьми', len(five_children))

Количество семей с пятью детьми 9


In [ ]:
grouped_by_children = data.pivot_table(index = 'children', values = 'debt', aggfunc= {'debt' :['mean', 'count']})
grouped_by_children.columns = ['количество детей', 'доля должников']
grouped_by_children

,количество детей,доля должников
children,,
0,14091.0,0.075438
1,4808.0,0.092346
2,2052.0,0.094542
3,330.0,0.081818
4,41.0,0.097561
5,9.0,0.000000


**Вывод:**

Таким образом, можно сделать вывод о том, что количество просрочек кредита напрямую завист от количество детей, то есть чем больше детей, тем меньше шанс банку получить возврат долга вовремя. Стоит учесть, что из-за маленького количества семей с пятью детьми нельзя брать эти данные для анализа.

#### 3.2 Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [ ]:
# Создаем сводную таблицу для того, чтобы выявить взаимосвязь между семейным положением и возвратом кредита в срок
family_and_kredit = data.pivot_table(columns = 'family_status', values = 'debt')
family_and_kredit

family_status,Не женат / не замужем,в разводе,вдовец / вдова,гражданский брак,женат / замужем
debt,0.097639,0.070648,0.066246,0.09313,0.075606


**Исправильно**

In [ ]:
grouped_by_family = data.pivot_table(index = 'family_status', values = 'debt', aggfunc= ['mean'])
grouped_by_family.columns = ['доля должников']
grouped_by_family

,доля должников
family_status,
Не женат / не замужем,0.097639
в разводе,0.070648
вдовец / вдова,0.066246
гражданский брак,0.093130
женат / замужем,0.075606


#### 3.3 Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [ ]:
# Создаем сводную таблицу для того, чтобы увидеть долю возврата кредита, для кажого уровня дохода
income_and_kredit = data.pivot_table(columns = 'total_income_category', values = 'debt')
income_and_kredit

total_income_category,A,B,C,D,E
debt,0.08,0.070621,0.085532,0.06,0.090909


In [ ]:
grouped_by_income = data.pivot_table(index = 'total_income_category', values = 'debt', aggfunc= ['mean'])
grouped_by_income.columns = ['доля должников']
grouped_by_income

,доля должников
total_income_category,
A,0.080000
B,0.070602
C,0.084982
D,0.060172
E,0.090909


0–30000 — 'E';
30001–50000 — 'D';
50001–200000 — 'C';
200001–1000000 — 'B';
1000001 и выше — 'A'

**Вывод:** Наименьшее число просроченных кредитов у людей с доходом от 50001 до 200000. Однако нельзя выявить взаимосвязь, что чем больше доход у людей, тем большая вероятность возврата кредита и наоборот.

#### 3.4 Как разные цели кредита влияют на его возврат в срок?

In [ ]:
# Создаем сводную таблицу для того, чтобы увидеть какие цели кредита влият на возврат кредита в срок.
purpose_and_kredit = data.pivot_table(columns = 'purpose_category', values = 'debt')
purpose_and_kredit

purpose_category,операции с автомобилем,операции с недвижимостью,получение образования,проведение свадьбы
debt,0.09348,0.072551,0.092528,0.079118


In [ ]:
grouped_by_purpose = data.pivot_table(index = 'purpose_category', values = 'debt', aggfunc= ['mean'])
grouped_by_purpose.columns = ['доля должников']
grouped_by_purpose

,доля должников
purpose_category,
операции с автомобилем,0.093480
операции с недвижимостью,0.072551
получение образования,0.092528
проведение свадьбы,0.079118


**Вывод:** Исходя из данных сводной таблицы, можно сделать вывод о том, что люди совершающие операции с недвижимостью и проведением свадьбы чаще всего возвращают долг в срок, в то время как люди совершающие операции с автомобилем и получением образования практически всегда задерживают возврат денег.

#### 3.5 Приведите возможные причины появления пропусков в исходных данных.

In [ ]:
# находим пропущенные значения в days_employed и total_income
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [ ]:
#выводим таблицу, где пропущены данные в столбцах, ищем взаимосвязь
skipped_values = data.loc[data['days_employed'].isna()]
skipped_values = data.loc[data['total_income'].isna()]
skipped_values.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
65,0,NaN,21,среднее,1,Не женат / не замужем,4,M,компаньон,0,NaN,операции с коммерческой недвижимостью
67,0,NaN,52,высшее,0,женат / замужем,0,F,пенсионер,0,NaN,покупка жилья для семьи
72,1,NaN,32,высшее,0,женат / замужем,0,M,госслужащий,0,NaN,операции с коммерческой недвижимостью
82,2,NaN,50,высшее,0,женат / замужем,0,F,сотрудник,0,NaN,жилье
83,0,NaN,52,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,жилье


*Ответ:* Изучив данные, можно увидеть, что пропуски в days_employed и total_income напрямую взаимосвязаны, поскольку пропуски в одинаковых строках. Также видно, что данные не могут быть пропущены из-за других значений в столбцах, поэтому можно сделать вывод о том, что пропуск данных может быть обусловлен некорректным заполнением данных.

#### 3.6 Объясните, почему заполнить пропуски медианным значением — лучшее решение для количественных переменных.

*Ответ:* Медина, позволяет получить такое значение, больше которого будет ровно половина значений, как и меньше. Медиана также корректно характеризует данные, значения которых сильно выделяются среди большинства и на это значение не влияют выбросы.

### Шаг 4: общий вывод.

В данном исследовании была осуществлена проверка разных факторов, которые влияют на возврат кредита вовремя.
Было рассмотрено четыре гипотезы, влияние колчества детей, семейного положения, уровня дохода и целей кредита.

Рассмотрим первую гипотезу:

In [ ]:
children_and_kredit

children,0,1,2,3,4,5
debt,0.075438,0.092346,0.094542,0.081818,0.097561,0.0


Она была частично подтверждена, поскольку не так много семей, обратившихся в банк, с 4-5 детьми.

Рассмотрим вторую гипотезу:

In [ ]:
family_and_kredit

family_status,Не женат / не замужем,в разводе,вдовец / вдова,гражданский брак,женат / замужем
debt,0.097639,0.070648,0.066246,0.09313,0.075606


Исходя из данных проверки взаимосвязи, можно сделать вывод о том, что люди ни разу не вступавшие в законный брак с большей вероятностью погашают кредит не вовремя.

Рассмотрим третью гипотезу:

In [ ]:
income_and_kredit

total_income_category,A,B,C,D,E
debt,0.08,0.070621,0.085532,0.06,0.090909


Данная гипотеза неможет быть подтверждена, в связи с тем, что есть пропущенные данные в доходе.

Рассмотрим четвертую гипотезу:

In [ ]:
purpose_and_kredit

purpose_category,операции с автомобилем,операции с недвижимостью,получение образования,проведение свадьбы
debt,0.09348,0.072551,0.092528,0.079118


Исходя из данных, сводной таблицы, можно сделать вывод о том, что то люди совершающие операции с недвижимостью и проведением свадьбы чаще всего возвращают долг в срок, в то время как люди совершающие операции с автомобилем и получением образования практически всегда задерживают возврат денег.